In [0]:
## IE-406 ML Project

In [0]:
## Using Google Colab for GPU usage
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive') 

Mounted at /content/drive


In [0]:
## Importing the necessary libraries
import os
import re
import nltk
import pandas as pd
import csv
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, f1_score
from sklearn.svm import LinearSVC

!pip install Unidecode
!pip install Word2number
from bs4 import BeautifulSoup
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import unidecode
from word2number import w2n
from pycontractions import Contractions
import gensim.downloader as api
from nltk.tokenize import TweetTokenizer
import string
import re
nltk.download('stopwords')
nltk.download('wordnet')

In [0]:
os.chdir('/content/drive/My Drive/as-9')

In [0]:
## Train data prep
train_set = pd.read_csv('train.tsv', delimiter='\t', encoding='utf-8')
train_list = list(train_set['text'])
train = train_set[train_set['task_1'] == 'HOF']
train_frame = {0:train_list,1:list(train['text']),2:list(train['text'])}
dict_train = {0:train_set['task_1'],1:train['task_2'],2:train['task_3']}

In [0]:
## Test data prep
test_set = pd.read_csv('test.tsv', delimiter='\t', encoding='utf-8')
test_list = list(test_set['text'])
test = test_set[test_set['task_1'] == 'HOF']
test_frame = {0:test_list,1:list(test['text']),2:list(test['text'])}
dict_test = {0:test_set['task_1'],1:test['task_2'],2:test['task_3']}

In [0]:
!sudo apt install openjdk-8-jdk
!sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!pip install language-check
!pip install pycontractions

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jre x11-utils
Suggested packages:
  openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jdk openjdk-8-jre x11-utils
0 upgraded, 8 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,676 kB/4,942 kB of archives.
After this operation, 13.3 MB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre amd64 8u242-b08-0ubuntu3~18.04
Ign:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jdk amd64 8u242-b08-0ubuntu3~18.04
Err:1 http://security.ubuntu.com/ubuntu bionic-updat

In [0]:
## Pre-trained Glove Embeddings
glove_model = api.load("glove-twitter-25")
contractions = Contractions(kv_model=glove_model)
contractions.load_models()
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
## Preprocessing
def pre(text):
    
    part = BeautifulSoup(text, "html.parser")
    text = part.get_text(separator=" ")
    text = text.strip()
    text = ' '.join(text.split())
    text = unidecode.unidecode(text)
    text = list(contractions.expand_texts([text], precise=True))[0]
    text = re.sub('[^a-zA-Z0-9 \n\.]', '', text)
    text = re.sub(r'\d', '',text)
    text = text.lower()

    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text) #tokenise text
    
    for w in tokens:
      if w in stop_words:
        tokens.remove(w)
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(i) for i in tokens]
    text = " ".join(tokens)
    return text

In [0]:
vocab = []
for data in train_list:
    for x in data.split(" "):
        vocab.append(x)
vocab = list(set(vocab))

for i in range(3):
    for j in range(len(train_frame[i])):
        train_frame[i][j] = pre(train_frame[i][j])
    for j in range(len(test_frame[i])):
        test_frame[i][j] = pre(test_frame[i][j])

In [0]:
count_vect = CountVectorizer(analyzer = 'word',vocabulary = vocab)
tfidf_transformer = TfidfTransformer()

In [0]:
## Naive Bayes Classifier
## We can use SGD/SVM or any other classifier here too
dict_train_2 = {}
dict_test_2 = {}
for i in range(3):
    count_train = count_vect.fit_transform(train_frame[i])
    tfidf_train = tfidf_transformer.fit_transform(count_train)
    count_test = count_vect.fit_transform(test_frame[i])
    tfidf_test = tfidf_transformer.fit_transform(count_test)
    le = LabelEncoder()
    le.fit(list(dict_train[i]))
    dict_train_2[i] = le.transform(dict_train[i].astype(str))
    cl_2 = MultinomialNB().fit(tfidf_train, dict_train_2[i])
    predictor = cl_2.predict(tfidf_test)
    dict_test_2[i] = le.transform(dict_test[i].astype(str))
    print(classification_report(dict_test_2[i], predictor))

              precision    recall  f1-score   support

           0       0.84      0.09      0.17       288
           1       0.77      0.99      0.87       865

    accuracy                           0.77      1153
   macro avg       0.81      0.54      0.52      1153
weighted avg       0.79      0.77      0.69      1153

              precision    recall  f1-score   support

           0       0.44      0.99      0.61       124
           1       0.00      0.00      0.00        71
           2       0.67      0.04      0.08        93

    accuracy                           0.44       288
   macro avg       0.37      0.34      0.23       288
weighted avg       0.40      0.44      0.29       288

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
## With Pretrained embeddings - GloVe
embed = {}
with open("pre-trained.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        v = np.asarray(values[1:], "float32")
        embed[word] = v

In [0]:
train_frame_2 = train_frame
test_frame_2 = test_frame
for j in range(3):
    for i in range(len(train_frame_2[j])):
        train_frame_2[j][i] = pre(train_frame_2[j][i])
        zero = np.zeros((1,100))
        for word in train_frame_2[j][i]:
            try:
                zero += embed[word]
            except:
                pass
        train_frame_2[j][i] = zero/len(train_frame_2[j][i])
    for i in range(len(test_frame_2[j])):
        test_frame_2[j][i] = pre(test_frame_2[j][i])
        zero = np.zeros((1,100))
        for word in test_frame_2[j][i]:
            try:
                zero += embed[word]
            except:
                pass
        test_frame_2[j][i] = zero/len(test_frame_2[j][i])
for j in range(3):
  train_frame_2[j] = np.array(train_frame_2[j]).reshape(-1,100)
  test_frame_2[j] = np.array(test_frame_2[j]).reshape(-1,100) 

In [0]:
dict_train_2=dict_train
dict_test_2=dict_test
for i in range(3):
    le = LabelEncoder()
    le.fit(list(dict_train[i]))
    dict_train_2[i] = le.transform(dict_train_2[i].astype(str))
    cl_3 = SGDClassifier().fit(train_frame_2[i], dict_train_2[i])
    predictor = cl_3.predict(test_frame_2[i])
    dict_test_2[i] = le.transform(dict_test_2[i].astype(str))
    print(classification_report(dict_test_2[i], predictor))

              precision    recall  f1-score   support

           0       0.26      1.00      0.41       288
           1       1.00      0.04      0.07       865

    accuracy                           0.28      1153
   macro avg       0.63      0.52      0.24      1153
weighted avg       0.81      0.28      0.16      1153

              precision    recall  f1-score   support

           0       0.50      0.84      0.63       124
           1       0.47      0.10      0.16        71
           2       0.61      0.44      0.51        93

    accuracy                           0.53       288
   macro avg       0.53      0.46      0.44       288
weighted avg       0.53      0.53      0.48       288

              precision    recall  f1-score   support

           0       0.86      0.93      0.89       245
           1       0.24      0.12      0.16        43

    accuracy                           0.81       288
   macro avg       0.55      0.53      0.53       288
weighted avg       0